In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import kraft

In [3]:
na_pa = {}

na_se_li_ = {}

for pa in kraft.path.get_child("../raw/"):

    if pa[-3:] != ".md":

        continue

    na = pa.split("/")[-1][: -len(".md")]

    assert na == kraft.string.untitle(na)

    na_pa[na] = pa

    se_ = [".", "<", ">"]

    se_li_ = {se: [] for se in se_}

    with open(pa) as io:

        for li in io:

            li = li.strip()

            if li == "":

                continue

            if li[0] == "#":

                se = li[2]

                assert se_.pop(0) == se

                continue

            # TODO: remove ()
            if li[0] == "(":

                assert li[-2] == "."

                # continue

            se_li_[se].append(li)

    na_se_li_[na] = se_li_

len(na_se_li_)

278

In [4]:
for na, se_li_ in na_se_li_.items():

    for li in se_li_["<"]:

        assert kraft.string.untitle(li) not in na_se_li_, "{}: {}".format(na, li)

In [5]:
import pandas as pd

In [6]:
fe_pr = pd.read_csv("../feature.tsv", "\t", index_col=0, squeeze=True).to_dict()

for fe, pr in fe_pr.items():

    fe_pr[fe] = str(pr)

In [7]:
for se_li_ in na_se_li_.values():

    se_li_["md<"] = []

    se_li_[">md"] = []

for na, se_li_ in na_se_li_.items():

    for li in list(se_li_[">"]):

        limi = kraft.string.untitle(li)

        if limi in na_se_li_:

            assert li not in fe_pr, li

            se_li_[">"].remove(li)

            se_li_[">md"].append(li)

            na_se_li_[limi]["md<"].append(na)

In [8]:
def combine_right(se_li_):

    return se_li_[">"] + kraft.iterable.flatten(
        [na_se_li_[kraft.string.untitle(li)][">"] for li in se_li_[">md"]]
    )

In [9]:
for na, se_li_ in na_se_li_.items():

    for li, n in pd.value_counts(combine_right(se_li_)).items():

        if 1 < n:

            so_ = [na]

            so_ += [
                liri
                for liri in se_li_[">md"]
                if li in na_se_li_[kraft.string.untitle(liri)][">"]
            ]

            print("{}:\n\t{}\n".format(li, "\n\t".join(so_)))

Tired:
	heart_failure
	Obstructive sleep apnea

Abdominal pain:
	diabetes_mellitus
	Diabetic ketoacidosis
	Gastroesophageal reflux disease

Nausea:
	diabetes_mellitus
	Diabetic ketoacidosis
	Chronic kidney disease

Tired:
	diabetes_mellitus
	Chronic kidney disease

Dehydrated:
	diabetes_mellitus_type_2
	Diabetes mellitus
	Diabetes mellitus
	Hyperosmolar hyperglycemic state

High blood osmolality:
	diabetes_mellitus_type_2
	Diabetes mellitus
	Diabetes mellitus
	Hyperosmolar hyperglycemic state

Hyperglycemia:
	diabetes_mellitus_type_2
	Diabetes mellitus
	Diabetes mellitus
	Hyperosmolar hyperglycemic state

Kidney disease:
	diabetes_mellitus_type_2
	Diabetes mellitus
	Diabetes mellitus

High blood hemoglobin A1c:
	diabetes_mellitus_type_2
	Diabetes mellitus
	Diabetes mellitus

Diarrhea:
	diabetes_mellitus_type_2
	Diabetes mellitus
	Diabetes mellitus

Hungry:
	diabetes_mellitus_type_2
	Diabetes mellitus
	Diabetes mellitus

Gangrene:
	diabetes_mellitus_type_2
	Diabetes mellitus
	Diabetes m

In [10]:
fe_ = []

for se_li_ in na_se_li_.values():

    fe_ += se_li_["<"] + se_li_[">"]

In [11]:
mi_ = []

for fe in fe_pr.keys():

    if fe not in fe_:

        mi_.append(fe)

mi_

['Breathing trouble that is sudden at night',
 'Breathing trouble',
 'Breathing trouble after bending',
 'Breathing trouble during excertion',
 'Breathing trouble when lying down',
 'Constipated',
 'Nauseated',
 'Panic attack',
 'Urine white blood cell',
 'High urine pH',
 'Low urine pH',
 'Stool with blood']

In [12]:
ne_ = []

for na, se_li_ in na_se_li_.items():

    for fe in se_li_["<"] + se_li_[">"]:

        if fe not in fe_pr:

            ne_.append([na, fe])

assert len(ne_) == 0, ne_

In [13]:
def number(pr):

    return [int(it) for it in pr.split(".")]


def prioritize(fe):

    if fe.endswith(".") or kraft.string.untitle(fe) in na_se_li_:

        return number("0.0")

    return number(fe_pr[fe])

In [14]:
de = "\n\n"

In [15]:
na_na2 = {}

In [16]:
if 0 < len(na_na2):

    for na, na2 in na_na2.items():

        fe_pr[na2] = fe_pr[na]

    se = pd.Series(fe_pr).sort_values(key=lambda se: se.map(number).values)

    se.index.name = "Feature"

    se.name = "Priority"

    se.to_csv("../feature.tsv", "\t")

    di = "../clean/"

    for na, se_li_ in na_se_li_.items():

        pa = na_pa[na].replace("../raw/", di)

        kraft.path.make(pa, pr=False)

        with open(na_pa[na], "w") as io:

            ti_li_ = {
                ".": [],
                "<": [],
                ">": [],
            }

            for li in se_li_["."]:

                ti_li_["."].append(li)

            for li in se_li_["<"]:

                ti_li_["<"].append(na_na2.get(li, li))

            for li in se_li_[">"] + se_li_[">md"]:

                ti_li_[">"].append(na_na2.get(li, li))

            for ti, li_ in ti_li_.items():

                io.write("# {}".format(ti) + de)

                if len(li_) == 0:

                    continue

                io.write(de.join(li_) + de)

In [17]:
import os
import shutil

In [18]:
di = "../compiled/"

if os.path.isdir(di):

    shutil.rmtree(di)

for na, se_li_ in na_se_li_.items():

    pa = na_pa[na].replace("../raw/", di)

    kraft.path.make(pa, pr=False)

    with open(pa, "w") as io:

        ti_li_ = {
            "Comment": [],
            "Eppremiology": [],
            "Cause": [],
            "Symptom": [],
            "Sign": [],
            "Diagnostic": [],
            # "Treatment and Management": [],
        }

        for li in se_li_["."]:

            ti_li_["Comment"].append(li)

        for li in se_li_["<"]:

            if number(fe_pr[li])[0] == 4:

                ti_li_["Eppremiology"].append(li)

        for li in se_li_["md<"]:

            ti_li_["Cause"].append(kraft.string.title(li))

        for li in combine_right(se_li_):

            if number(fe_pr[li])[0] == 1:

                ti_li_["Symptom"].append(li)

            elif number(fe_pr[li])[0] == 6:

                ti_li_["Sign"].append(li)

            elif 7 <= number(fe_pr[li])[0]:

                ti_li_["Diagnostic"].append(li)

        for ti, li_ in ti_li_.items():

            io.write("# {}".format(ti) + de)

            if len(li_) == 0:

                continue

            io.write(de.join(sorted(li_, key=prioritize)) + de)